<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading 

**Chapter 07 &mdash; Working with Real-Time Data and Sockets**

## Visualizing Streaming Data with Plotly

### The Basics

In [1]:
import zmq
from datetime import datetime
import plotly.graph_objects as go

In [2]:
symbol = 'SYMBOL'

In [3]:
fig = go.FigureWidget()
fig.add_scatter()
fig

FigureWidget({
    'data': [{'type': 'scatter', 'uid': '72048da0-b0cd-431d-9967-953748fdf6d9'}], 'layout': {'template': '...'}
})

In [4]:
context = zmq.Context()

In [5]:
socket = context.socket(zmq.SUB)

In [6]:
socket.connect('tcp://0.0.0.0:5555')

<SocketContext(connect='tcp://0.0.0.0:5555')>

In [7]:
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

In [8]:
times = list()
prices = list()

In [9]:
for _ in range(50):
    msg = socket.recv_string()
    t = datetime.now()
    times.append(t)
    _, price = msg.split()
    prices.append(float(price))
    fig.data[0].x = times
    fig.data[0].y = prices

In [10]:
fig = go.FigureWidget()
fig.add_scatter(name='SYMBOL')
fig.add_scatter(name='SMA1', line=dict(width=1, dash='dot'),
                mode='lines+markers')
fig.add_scatter(name='SMA2', line=dict(width=1, dash='dash'),
                mode='lines+markers')
fig

FigureWidget({
    'data': [{'name': 'SYMBOL', 'type': 'scatter', 'uid': 'be5d42ab-e9a8-4aa8-b8ad-cf415044e458'},
             {'line': {'dash': 'dot', 'width': 1},
              'mode': 'lines+markers',
              'name': 'SMA1',
              'type': 'scatter',
              'uid': '8f4df55d-899d-4ac4-afb4-a8973051e8e6'},
             {'line': {'dash': 'dash', 'width': 1},
              'mode': 'lines+markers',
              'name': 'SMA2',
              'type': 'scatter',
              'uid': 'b3b21479-fec2-4f63-b716-e90fa4ff6c7a'}],
    'layout': {'template': '...'}
})

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame()

In [13]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = pd.concat((df, pd.DataFrame({sym: float(price)}, index=[t])))
    df['SMA1'] = df[sym].rolling(5).mean()
    df['SMA2'] = df[sym].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['SMA1']
    fig.data[2].y = df['SMA2']

In [14]:
from plotly.subplots import make_subplots

In [15]:
f = make_subplots(rows=3, cols=1, shared_xaxes=True)
f.append_trace(go.Scatter(name='SYMBOL'), row=1, col=1)
f.append_trace(go.Scatter(name='RETURN', line=dict(width=1, dash='dot'),
                mode='lines+markers', marker={'symbol': 'triangle-up'}),
                row=2, col=1)
f.append_trace(go.Scatter(name='MOMENTUM', line=dict(width=1, dash='dash'),
                mode='lines+markers', marker={'symbol': 'x'}), row=3, col=1)
# f.update_layout(height=600)

In [16]:
fig = go.FigureWidget(f)

In [17]:
fig

FigureWidget({
    'data': [{'name': 'SYMBOL',
              'type': 'scatter',
              'uid': 'f9a86409-0e20-4ee3-a8d9-56e32463e611',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'line': {'dash': 'dot', 'width': 1},
              'marker': {'symbol': 'triangle-up'},
              'mode': 'lines+markers',
              'name': 'RETURN',
              'type': 'scatter',
              'uid': 'dc707b4b-85a4-4473-b02e-aa6d6dd601e6',
              'xaxis': 'x2',
              'yaxis': 'y2'},
             {'line': {'dash': 'dash', 'width': 1},
              'marker': {'symbol': 'x'},
              'mode': 'lines+markers',
              'name': 'MOMENTUM',
              'type': 'scatter',
              'uid': '9dd7b5d2-3243-4aea-8e11-e3a2883edf9a',
              'xaxis': 'x3',
              'yaxis': 'y3'}],
    'layout': {'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'matches': 'x3', 'showticklabels': False},
               'x

In [18]:
import numpy as np

In [19]:
df = pd.DataFrame()

In [20]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = pd.concat((df, pd.DataFrame({sym: float(price)}, index=[t])))
    df['RET'] = np.log(df[sym] / df[sym].shift(1))
    df['MOM'] = df['RET'].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['RET']
    fig.data[2].y = df['MOM']

In [21]:
socket = context.socket(zmq.SUB)

In [22]:
socket.connect('tcp://0.0.0.0:5556')

<SocketContext(connect='tcp://0.0.0.0:5556')>

In [23]:
socket.setsockopt_string(zmq.SUBSCRIBE, '')

In [24]:
for _ in range(5):
    msg = socket.recv_string()
    print(msg)

48.674 40.853 24.742 27.366 64.037 1.399 32.981 18.008
70.966 36.151 78.019 57.747 56.173 77.009 99.554 91.190
93.148 47.537 13.005 48.135 31.320 45.457 71.659 30.417
30.169 38.787 32.575 93.062 51.436 72.789 74.867 73.174
99.071 18.374 62.666 23.443 52.937 52.289 34.678 40.462


In [25]:
fig = go.FigureWidget()
fig.add_bar()
fig

FigureWidget({
    'data': [{'type': 'bar', 'uid': '359463cd-7a97-42a3-8459-bb67201b11f2'}], 'layout': {'template': '...'}
})

In [26]:
x = list('abcdefgh')
fig.data[0].x = x
for _ in range(100):
    msg = socket.recv_string()
    y = msg.split()
    y = [float(n) for n in y]
    fig.data[0].y = y

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>